In [ ]:
#Overarching importation
import sys, os
PROJECT_ROOT = os.path.abspath("..")  # go up one directory from notebooks/
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

In [ ]:
from src.Data_ingestor import DataIngestorFactory
from src.Missing_value_handling import MissingValueHandler,FillMissingValue,DropMissingValue

#-------#
# To help with reading and manipulating data
import pandas as pd
import numpy as np

# To help with data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# To be used for missing value imputation
from sklearn.impute import SimpleImputer

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To get different metric scores, and split data
from sklearn import metrics
#---#
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score
)

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To be used for tuning the model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMClassifier

# To use statistical functions
import scipy.stats as stats

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
import warnings

warnings.filterwarnings("ignore")

In [ ]:
#Load Data
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"

In [ ]:
#Load Data
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"
df_train = DataIngestorFactory.get_data_ingestor(TRAIN_ENDPOINT).ingest(TRAIN_ENDPOINT)
df_test = DataIngestorFactory.get_data_ingestor(TEST_ENDPOINT).ingest(TEST_ENDPOINT)

In [ ]:
handler = MissingValueHandler(FillMissingValue,method = 'mean')

handler.handle_missing_values(df_train)

In [ ]:
from src.Data_Samplier import SamplerFactory

samplier = SamplerFactory.create('smoteenn')
df_scaled = samplier.impute(df_train,target_col= 'Target')

In [ ]:
#If we want to scale up Trainig Set
from src.Data_Samplier import SMOTEENNSampler
df_scaled = SMOTEENNSampler().impute(df_train,target_col='Target')
df_scaled['Target'].value_counts()

In [ ]:
from src.Model_Selector import CrossValidationEvaluation,ModelEvaluator

In [ ]:
X = df_scaled.drop('Target', axis=1)
y = df_scaled['Target']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2,     
    random_state=42,    
    stratify=y           
)


In [ ]:
# #Model Selection
models = [
    ("log", LogisticRegression(solver="newton-cg", random_state=42)),
    ("DecisionTreeClassifier", DecisionTreeClassifier(random_state=42)),
    ("XGBClassifier", XGBClassifier(random_state=42, eval_metric="logloss", device='cpu'))
]

In [ ]:
# Step 1: create evaluation strategy
strategy = CrossValidationEvaluation(n_splits=5, random_state=42)

# Step 2: create the evaluator with that strategy
evaluator = ModelEvaluator(strategy=strategy)

# Step 3: evaluate  models
results = evaluator.evaluate_models(models=models, X=X_train, y=y_train)

# Step 4: Return best model name and best model attribute
best_model_name,best_model = evaluator.get_best_model(models)


In [ ]:
#Model Tunning
from src.Model_Tuner import OptunaTuning

#Step 1 Define Tuning configuration
tuner = OptunaTuning(config_dir="config", n_trials=50, cv_folds=5)

#Step 2 Run Tuning to get final model, parameter and best score
best_model, best_params, best_score = tuner.tune("xgbclassifier", XGBClassifier, X_train, y_train)


In [ ]:
tuner.save_tuned_model("xgbclassifier", best_model)

In [ ]:
from src.Model_Evaluator import ClassificationModelEvaluator

In [ ]:
eval_map = {
        "classification": '123'
       # "regression": 
}